In [1]:
import os

In [2]:
%pwd

'/Users/ivanosipchyk/dev/studies/end-to-end-ml-project/research'

In [3]:
os.chdir('../')

In [4]:
!pwd

/Users/ivanosipchyk/dev/studies/end-to-end-ml-project


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    mlflow_uri: str


In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.SVC

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            mlflow_uri='https://dagshub.com/ivan.osipchyk.work/end-to-end-ml-project.mlflow'
        )

        return model_evaluation_config

In [25]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import joblib

In [42]:
from mlProject.utils.common import save_json

class ModelEvaluation:
    def __init__(self, config):
        self.config = config
        mlflow.set_tracking_uri(config.mlflow_uri)

    def eval_metrics(self, y_true, y_pred):
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='weighted')
        recall = recall_score(y_true, y_pred, average='weighted')
        f1 = f1_score(y_true, y_pred, average='weighted')
        return accuracy, precision, recall, f1

    def log_into_mlflow(self):
        X_test = pd.read_csv(os.path.join(self.config.data_path, 'x_test.csv'), skiprows=1, header=None)
        y_test = pd.read_csv(os.path.join(self.config.data_path, 'y_test.csv'))

        model = joblib.load(self.config.model_path)

        with mlflow.start_run():
            y_pred = model.predict(X_test)

            acc, pr, rec, f1 = self.eval_metrics(y_test, y_pred)

            # Log metrics to mlflow
            mlflow.log_metric('accuracy', acc)
            mlflow.log_metric('precision', pr)
            mlflow.log_metric('recall', rec)
            mlflow.log_metric('f1 score', f1)

            # Log parameters to mlflow
            mlflow.log_params(self.config.all_params)

            # Save metrics to a JSON file using save_json utility
            scores = {
                'accuracy': acc,
                'precision': pr,
                'recall': rec,
                'f1 score': f1
            }
            save_json(path=Path(os.path.join(self.config.root_dir, self.config.metric_file_name)), data=scores)

            # Log the model to mlflow
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, 'model', registered_model_name='SVC')
            else:
                mlflow.sklearn.log_model(model, "model")

In [45]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    model_eval_config = ModelEvaluation(config=model_eval_config)
    model_eval_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-07-09 14:38:04,734: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-09 14:38:04,735: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-09 14:38:04,737: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-09 14:38:04,738: INFO: common: created directory at: artifacts]
[2024-07-09 14:38:04,739: INFO: common: created directory at: artifacts/model_evaluation]


/Users/ivanosipchyk/dev/studies/end-to-end-ml-project/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/ivanosipchyk/dev/studies/end-to-end-ml-project/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[2024-07-09 14:38:06,521: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


Registered model 'SVC' already exists. Creating a new version of this model...
2024/07/09 14:38:11 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: SVC, version 3
Created version '3' of model 'SVC'.


In [13]:
X_test = pd.read_csv(os.path.join('artifacts/data_transformation', 'x_test.csv'))

In [14]:
X_test.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,2.719067,0.615,0.66,1.837424,-2.340266,2.043721,4.109736,1.00220,3.07,-0.312246,2.440369
1,2.473559,0.270,0.24,0.755871,-2.465357,2.899951,3.820764,0.99560,3.22,-0.411227,2.719067
2,2.090022,0.735,0.00,0.804206,-2.372742,3.109664,3.625875,0.99765,3.41,-0.504356,2.371051
3,2.232275,0.340,0.40,0.894913,-2.554798,3.444478,4.271245,0.99554,3.34,-0.150254,2.547564
4,2.607519,0.440,0.64,0.755871,-2.639071,1.675968,2.899951,0.99800,3.21,-0.396493,2.462599
